<a href="https://colab.research.google.com/github/LejoFlores/CdA_WatershedAnalysis/blob/main/CdA_PRISM_Clip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
try:
    import wxee
except ImportError:
    print('wxee package not installed. Installing...') 
    subprocess.check_call(['python','-m','pip','install','wxee'])
  #!pip install eemont

import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

from google.colab import drive
drive.mount('/content/gdrive')

import wxee

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
prism_write_path = '/content/gdrive/My Drive/_CdA_GEE_data/PRISM_data/'

cda_ws_norm_nc = 'prism_cda_watershed_1981-2010_norm.nc'

up_cda_norm_nc = 'prism_up_cda_1981-2010_norm.nc'
sf_cda_norm_nc = 'prism_sf_cda_1981-2010_norm.nc'
lk_cda_norm_nc = 'prism_lk_cda_1981-2010_norm.nc'
st_joe_norm_nc = 'prism_st_joe_1981-2010_norm.nc'

cda_ws_mon_nc = 'prism_cda_watershed_1991-2020_mon.nc'

up_cda_mon_nc = 'prism_up_cda_1991-2020_mon.nc'
sf_cda_mon_nc = 'prism_sf_cda_1991-2020_mon.nc'
lk_cda_mon_nc = 'prism_lk_cda_1991-2020_mon.nc'
st_joe_mon_nc = 'prism_st_joe_1991-2020_mon.nc'


In [3]:
# Get the USGS HUC08 collection from the data catalog
huc08 = ee.FeatureCollection('USGS/WBD/2017/HUC08')

# Get the three basins within the Northwest that correspond to the Lake Coeur d'Alene Watershed
# Find the correspondence between the basin name and codes here: https://water.usgs.gov/GIS/huc.html

# 4-Digit HUCs:
# 17010301 -- Upper Coeur d'Alene. Idaho. Area = 905 sq.mi.
# 17010302 -- South Fork Coeur d'Alene. Idaho. Area = 297 sq.mi.
# 17010303 -- Coeur d'Alene Lake. Idaho, Washington. Area = 663 sq.mi.
# 17010304 -- St. Joe. Idaho. Area = 1860 sq.mi.

up_cda_watershed = huc08.filter(ee.Filter.inList('huc8', ee.List(['17010301'])))
sf_cda_watershed = huc08.filter(ee.Filter.inList('huc8', ee.List(['17010302'])))
lk_cda_watershed = huc08.filter(ee.Filter.inList('huc8', ee.List(['17010303'])))
st_joe_watershed = huc08.filter(ee.Filter.inList('huc8', ee.List(['17010304'])))

# Get the whole watershed
cda_watershed = huc08.filter(ee.Filter.inList('huc8', ee.List(['17010301','17010302','17010303','17010304'])))

# Now union the three basins to get one feature that consists of the entire Snake River
cda_watershed_full = cda_watershed.union()

# Create a new map and add the watershed we will use to clip the EVI data
Map = emap.Map(center=(47.5, -116.25),zoom=9)

Map.addLayer(ee.Image().paint(up_cda_watershed, 0, 2), {}, 'Upper Coeur d\'Alene River')
Map.addLayer(ee.Image().paint(sf_cda_watershed, 0, 2), {}, 'South Fork Coeur d\'Alene River')
Map.addLayer(ee.Image().paint(lk_cda_watershed, 0, 2), {}, 'Lake Coeur d\'Alene River')
Map.addLayer(ee.Image().paint(st_joe_watershed, 0, 2), {}, 'St. Joe River')

Map.addLayerControl()
Map

In [ ]:
ds_cda = ee.ImageCollection('OREGONSTATE/PRISM/Norm81m').map(lambda img: img.clip(cda_watershed_full)).wx.to_xarray(path=prism_write_path+cda_ws_norm_nc, region=cda_watershed_full.geometry(), scale=4000)
ds_up_cda = ee.ImageCollection('OREGONSTATE/PRISM/Norm81m').map(lambda img: img.clip(up_cda_watershed)).wx.to_xarray(path=prism_write_path+up_cda_norm_nc,region=up_cda_watershed.geometry(), scale=4000)
ds_sf_cda = ee.ImageCollection('OREGONSTATE/PRISM/Norm81m').map(lambda img: img.clip(sf_cda_watershed)).wx.to_xarray(path=prism_write_path+sf_cda_norm_nc,region=sf_cda_watershed.geometry(), scale=4000)
ds_lk_cda = ee.ImageCollection('OREGONSTATE/PRISM/Norm81m').map(lambda img: img.clip(lk_cda_watershed)).wx.to_xarray(path=prism_write_path+lk_cda_norm_nc,region=lk_cda_watershed.geometry(), scale=4000)
ds_st_joe = ee.ImageCollection('OREGONSTATE/PRISM/Norm81m').map(lambda img: img.clip(st_joe_watershed)).wx.to_xarray(path=prism_write_path+st_joe_norm_nc,region=st_joe_watershed.geometry(), scale=4000)

mon_ds_cda = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').filter(ee.Filter.date('1991-01-01','2020-12-31')).map(lambda image: image.clip(cda_watershed_full)).wx.to_xarray(path=prism_write_path+cda_ws_mon_nc, region=cda_watershed_full.geometry(), scale=4000)
mon_ds_up_cda = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').filter(ee.Filter.date('1991-01-01','2020-12-31')).map(lambda image: image.clip(up_cda_watershed)).wx.to_xarray(path=prism_write_path+up_cda_mon_nc, region=up_cda_watershed.geometry(), scale=4000)
mon_ds_sf_cda = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').filter(ee.Filter.date('1991-01-01','2020-12-31')).map(lambda image: image.clip(sf_cda_watershed)).wx.to_xarray(path=prism_write_path+sf_cda_mon_nc, region=sf_cda_watershed.geometry(), scale=4000)
mon_ds_lk_cda = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').filter(ee.Filter.date('1991-01-01','2020-12-31')).map(lambda image: image.clip(lk_cda_watershed)).wx.to_xarray(path=prism_write_path+lk_cda_mon_nc, region=lk_cda_watershed.geometry(), scale=4000)
mon_ds_st_joe = ee.ImageCollection('OREGONSTATE/PRISM/AN81m').filter(ee.Filter.date('1991-01-01','2020-12-31')).map(lambda image: image.clip(st_joe_watershed)).wx.to_xarray(path=prism_write_path+st_joe_mon_nc, region=st_joe_watershed.geometry(), scale=4000)


Requesting data:   0%|          | 0/12 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/12 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/12 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/12 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/12 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/360 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/360 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/360 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/360 [00:00<?, ?it/s]

Requesting data:   0%|          | 0/360 [00:00<?, ?it/s]

In [ ]:
ds_sf_cda.to_netcdf(prism_write_path+'test2.nc')